In [ ]:
from torch.nn.functional import affine_grid
import monai
from monai.transforms import (
    AddChanneld,
    LoadImaged,
    ToTensord,
)
import torch as t
from copy import deepcopy
import torchio as tio
import numpy as np

In [ ]:
path = "sample_data/10_3T_nody_002.nii.gz"
add_channel = AddChanneld(keys=["image"])
loader = LoadImaged(keys=["image"])
to_tensor = ToTensord(keys=["image"])
monai_dict = {"image": path}
monai_image_layer = loader(monai_dict)
monai_image_layer = add_channel(add_channel(monai_image_layer))
monai_image_layer = to_tensor(monai_image_layer)

In [ ]:
image_tensor = monai_image_layer["image"]
input_2d = image_tensor[:,:,:,:,12]
input_2d.shape

In [ ]:
theta = t.eye(3)[:2,:].unsqueeze(0)
theta[0,0,2] = 1
size = input_2d.shape
align_corners = False
grid = affine_grid(theta,size, align_corners=align_corners)

In [ ]:
output_2d = monai.networks.layers.grid_pull(input_2d,grid,interpolation = 'cubic', bound = "zero", extrapolate = False)

In [ ]:
theta.shape

In [ ]:
gaussain_smoother = monai.transforms.GaussianSmooth(sigma=0.2)

golay_filter = monai.transforms.SavitzkyGolaySmooth(4, 4, axis=1, mode='zeros')

tio_image = tio.Image("10_3T_nody_002.nii.gz")
smoothed_tensor = golay_filter(tio_image.tensor)
tio_smoothed = tio.Image(tensor=smoothed_tensor, affine=tio_image.affine)